# Hi-Hello Training
한 문자를 주면 그 다음 문자가 무엇인지 예측하도록 만들기  
같은 문자(h)를 주어도 어떤 경우는 다음이 i이고, 어떤 경우는 다음이 e이다.  
이전 입력이 무엇인지 알아야 예측이 가능한 구조이므로, RNN으로 해결해야 한다.  

In [1]:
# Lab 12 RNN
import tensorflow as tf
###Disable Warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import logging
tf.get_logger().setLevel(logging.ERROR)
#import sys
#original_stdout = sys.stdout
#sys.stdout = None
#import warnings
#warnings.filterwarnings("ignore")

import numpy as np
tf.set_random_seed(777)  # reproducibility

#각각의 문자를 뽑아내고 원핫인코딩 시킨다.
#중복되는것을 제외한 유일한 문자(열)을 뽑아낸다. 이것 vochabulary or voc라고 부름
idx2char = ['h', 'i', 'e', 'l', 'o'] #뽑아낸 문자(열)에 인덱스를 붙인다.
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3 --> voc index. dictionary(dic)라고도 부른다

#=> input_dim = 5

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello #입력에 따른 올바른 출력

num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

X = tf.placeholder(
    tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
#Y는 Label 값이 들어가야 하므로 one_hot 불필요

#cell은 BasicRNNCell(), BasicLSTMCell, GRUCell 등 다양
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True) 
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32) 
    #output의 shape는 (batch, sequence_length, hidden_size)이다.

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length]) #weight는 1로 초기화(각 batch별로 각 sequence의 중요도 정의) => 모든 sequence의 중요도를 일정하게 보는 것임
sequence_loss = tf.contrib.seq2seq.sequence_loss( #sequence_loss api로 loss 구현
    logits=outputs, targets=Y, weights=weights) #logit은 어떤 확률과 연관되는 인자이므로
            #outputs를 준다(결과가 probability임. e.g. [0.1, ..., 0.6])
    #sequence_loss는 예측이 강한 쪽이 loss가 낮게 나온다. 
    #e.g. [0.1, 0.9]가 [0.3, 0.7]보다 loss가 작다.
    #sequence_loss()는 각각의 sequence에서 cross_entropy를 구하는 것과 같다.
    #다만, sequence는 여러개이고 cross_entropy를 모두 구현하면 코드가 너무 길고 복잡해진다.
    #따라서 sequence_loss()는 이걸 대신해준다.
    #복습: Cross-entropy는 예측과 실제 답의 "cross"를 통해 정답이 맞았을 확률을 구한다.
    #예측을 잘했다면 cross-entropy는 최대가 된다.
loss = tf.reduce_mean(sequence_loss) #각각의 sequence에서 sequence loss를 계산했으면 평균처리해주고 optimizer에 넘긴다.
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) #loss를 이용해 Weight들 업데이트

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)] #squeeze는 0d array로 찍어누르는 메소드. result를 가장 간단한 array로 찍어누른다음 모든 인자에 대해서 해당하는 문자를 찾아서 문자 배열로 만들어준다.
        print("\tPrediction str: ", ''.join(result_str)) #문자배열을 join메소드를 이용해 문자열로 만들기

'''
0 loss: 1.71584 prediction:  [[2 2 2 3 3 2]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelle
1 loss: 1.56447 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
2 loss: 1.46284 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
3 loss: 1.38073 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
4 loss: 1.30603 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
5 loss: 1.21498 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
6 loss: 1.1029 prediction:  [[3 0 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lhlllo
7 loss: 0.982386 prediction:  [[1 0 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihlllo
8 loss: 0.871259 prediction:  [[1 0 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihlllo
9 loss: 0.774338 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihello
10 loss: 0.676005 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihello

'''
pass
# 처음에는 prediction이 엉망으로 보이지만, 학습이 반복될수록 원하는 결과에 가까워진다.
pass

0 loss: 1.6078763 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
Tensor("sequence_loss/div_no_nan:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
1 loss: 1.5102621 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
Tensor("sequence_loss/div_no_nan:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
2 loss: 1.4327029 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
Tensor("sequence_loss/div_no_nan:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
3 loss: 1.3489527 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
Tensor("sequence_loss/div_no_nan:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
4 loss: 1.2551296 prediction:  [[1 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  illlll
Tensor("sequence_loss/div_no_nan:0", shape=(), dtype=float32)
Tensor("Mean